In [6]:
import os
import urllib3
import json
import time
import base64
import moviepy.editor as mp
from pydub import AudioSegment

openApiURL = "http://aiopen.etri.re.kr:8000/VideoParse" #API호출
accessKey = "16f35e3e-b273-4953-9366-4c7d68128530" #API접근 Key
path = r'C:\Users\skanj\Desktop\video'
video = '\movie4.mp4'
videoFilePath =  path + video

file = open(videoFilePath,'rb')#파일 열기 (이진파일 읽기 권한부여)
fileContent = file.read()
file.close()

requestJson = {
  "access_key": accessKey,
  "argument":{}
}

http = urllib3.PoolManager()

response = http.request(
  "POST",
  openApiURL,
  fields={
    'json': json.dumps(requestJson),
    'uploadfile': (os.path.basename(file.name), fileContent)
  }
)

print("[responseCode]" + str(response.status))
print("[responseBody]")
print(response.data)

time.sleep(30)

URL = openApiURL + '/status'
file_id = json.loads(response.data.decode())["return_object"]["file_id"]

requestJson1 = {
  "access_key": accessKey,
  "argument":{"file_id": file_id}
}

http1 = urllib3.PoolManager()

response1 = http1.request(
  "POST",
  URL,
  fields={
    'json': json.dumps(requestJson1),
    'uploadfile': (os.path.basename(file.name), fileContent)
  }
)

print("[responseCode]" + str(response1.status))
print("[responseBody]")
print(response1.data)


# audio path, language, API
languageCode = "korean"
audio = "\movie4.wav"
audioFilePath = path + audio
openApiURL = "http://aiopen.etri.re.kr:8000/WiseASR/Recognition"

# movie to audio
clip = mp.VideoFileClip(videoFilePath)
clip.audio.write_audiofile(audioFilePath)

# audio 
sound = AudioSegment.from_file(audioFilePath)

# 일단 앞에 5초만 나눠봄 (5초가 5000임)
split_point = 5000
splits = sound[:split_point]

# create a new split audio file 
split_audio_name = "\s1.wav"
split_audio_path = path + split_audio_name

# audio split
splits.export(split_audio_path, format="wav")

# 음성인식 API 시작 앞에 5초만 나눈 오디오로 함.
file = open(split_audio_path, "rb")
audioContents = base64.b64encode(file.read()).decode("utf8")
file.close()
 
requestJson = {
    "access_key": accessKey,
    "argument": {
        "language_code": languageCode,
        "audio": audioContents
    }
}
 
http = urllib3.PoolManager()
response = http.request(
    "POST",
    openApiURL,
    headers={"Content-Type": "application/json; charset=UTF-8"},
    body=json.dumps(requestJson)
)
 
print("[responseCode] " + str(response.status))
print("[responBody]")
print(str(response.data,"utf-8"))


[responseCode]200
[responseBody]
b'{"result":0,"return_object":{"file_id":"bec994ad7fe09485c276fefb498fc3ed.mp4"}}'
[responseCode]200
[responseBody]
b'{"result":0,"return_object":{"result":[{"num":6.0,"time":[0.0,56.0,76.0,81.0,92.0,99.0],"frame":[0.0,1414.0,1918.0,2046.0,2308.0,2499.0],"code":3.0,"msg":"All processes are done"}]}}'


chunk:   0%|                                                                        | 0/2432 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in C:\Users\skanj\Desktop\video\movie4.wav


MoviePy - Done.
[responseCode] 200
[responBody]
{"result":0,"return_object":{"recognized":"으 응 "}}
